In [76]:
import numpy as np
import os
import pandas as pd
from jinja2 import Template
import subprocess
from varname import nameof
import multiprocessing
# import sqlite3
# import glob
# import openpyxl
# import itertools
from scipy.stats import norm
import matplotlib.pyplot as plt

from functools import partial

In [129]:
DF_1_mults = [1.176214,   # 'FIXDGm_0'
         0.832719,   # 'FIXDGm_1'
         0.934867,   # 'FIXDGm_2'
         1.020313,   # 'FIXDGm_3'
         1.04418,   # 'FIXDGm_4'
         0.66636,   # 'MORTm_0'
         1.390145,   # 'MORTm_1'
         1.002098,   # 'MORTm_2'
         0.782002,   # 'MORTm_3'
         0.928679,   # 'MORTm_4'
         508.80522,   # 'SDI_MAX'
]

DF_2_mults = [0.975456,   # 'FIXDGm_0'
         0.681072,   # 'FIXDGm_1'
         1.122564,   # 'FIXDGm_2'
         0.915876,   # 'FIXDGm_3'
         0.843329,   # 'FIXDGm_4'
         1.235499,   # 'MORTm_0'
         0.752621,   # 'MORTm_1'
         1.100263,   # 'MORTm_2'
         0.928208,   # 'MORTm_3'
         0.73039,   # 'MORTm_4'
         508.80522,   # 'SDI_MAX'
] 

DF_3_mults = [1.171961,   # 'FIXDGm_0'
         1.065488,   # 'FIXDGm_1'
         1.192227,   # 'FIXDGm_2'
         1.260282,   # 'FIXDGm_3'
         0.765391,   # 'FIXDGm_4'
         0.8029,   # 'MORTm_0'
         0.974715,   # 'MORTm_1'
         0.825551,   # 'MORTm_2'
         0.891001,   # 'MORTm_3'
         1.069205,   # 'MORTm_4'
         798.602176,   # 'SDI_MAX'
] 

high_growth = [1.25,   # 'FIXDGm_0'
         1.25,   # 'FIXDGm_1'
         1.25,   # 'FIXDGm_2'
         1.25,   # 'FIXDGm_3'
         1.25,   # 'FIXDGm_4'
         0.75,   # 'MORTm_0'
         0.75,   # 'MORTm_1'
         0.75,   # 'MORTm_2'
         0.75,   # 'MORTm_3'
         0.75,   # 'MORTm_4'
         650,   # 'SDI_MAX'
]

high_mort = [0.75,   # 'FIXDGm_0'
         0.75,   # 'FIXDGm_1'
         0.75,   # 'FIXDGm_2'
         0.75,   # 'FIXDGm_3'
         0.75,   # 'FIXDGm_4'
         1.25,   # 'MORTm_0'
         1.25,   # 'MORTm_1'
         1.25,   # 'MORTm_2'
         1.25,   # 'MORTm_3'
         1.25,   # 'MORTm_4'
         650,   # 'SDI_MAX'
]

super_high_mort = [0.5,   # 'FIXDGm_0'
         0.5,   # 'FIXDGm_1'
         0.5,   # 'FIXDGm_2'
         0.5,   # 'FIXDGm_3'
         0.5,   # 'FIXDGm_4'
         1.75,   # 'MORTm_0'
         1.75,   # 'MORTm_1'
         1.75,   # 'MORTm_2'
         1.75,   # 'MORTm_3'
         1.75,   # 'MORTm_4'
         650,   # 'SDI_MAX'
]

default = [1,   # 'FIXDGm_0'
         1,   # 'FIXDGm_1'
         1,   # 'FIXDGm_2'
         1,   # 'FIXDGm_3'
         1,   # 'FIXDGm_4'
         1,   # 'MORTm_0'
         1,   # 'MORTm_1'
         1,   # 'MORTm_2'
         1,   # 'MORTm_3'
         1,   # 'MORTm_4'
         650,   # 'SDI_MAX'
]

All_1_mults = [1.087440, 0.937351, 0.966504, 0.970860, 1.136502, 0.778377, 1.192149, 
               1.372026, 1.114995, 0.911335, 578.684165]

All_2_mults = [1.050615, 1.18539448, 1.15936948, 0.75350904, 0.7758883,
               1.28498727, 1.32557039, 0.99596719, 0.97872349, 0.86110373, 530.69502948]

All_3_mults = [1.024421, 0.851047, 0.959006, 0.889773, 0.712680, 1.134344, 1.013402, 
               1.171479, 0.760650, 0.997969, 486.526700]

In [130]:
def make_param_dict(params, species, stand_id):
    ''' Single species
        FIXDG (diameter growth multiplier by species and diameter class)
        MORTMULT (background mortality rate multiplier by species and diameter class)
        SDIMAX (density-driven mortality parameter set for each species)
    '''
    
    param_dict = {}
    run_name = "super_high_mort"
    param_dict['run_name'] = run_name
    #stand input
    param_dict['stand_id'] = stand_id
    
    #input and output databases
    param_dict['FVSIn'] = '../data/FVSIn.db' 
    outpath = '../keyfiles/{}'.format(run_name)
    os.makedirs(outpath, exist_ok=True)
    param_dict['FVSOut'] = '../keyfiles/{}/{}_all.db'.format(run_name, run_name) ### Change name of database
    #param_dict['FVSOut'] = '../keyfiles/{}.db'.format(run_id)

    param_dict['species']  = species

    #growth factors
    #size classes: 3-5, 5-10, 10-15, 15-20, >20
    param_dict['FIXDGm_0'] = round(params[0],8)
    param_dict['FIXDGm_1'] = round(params[1],8)
    param_dict['FIXDGm_2'] = round(params[2],8)
    param_dict['FIXDGm_3'] = round(params[3],8)
    param_dict['FIXDGm_4'] = round(params[4],8)
    #background mortality factors
    param_dict['MORTm_0'] = round(params[5],8)
    param_dict['MORTm_1'] = round(params[6],8)
    param_dict['MORTm_2'] = round(params[7],8)
    param_dict['MORTm_3'] = round(params[8],8)
    param_dict['MORTm_4'] = round(params[9],8)

    #density driven mortality factors
    param_dict['SDI_MAX'] = int(params[10])
    # param_dict['SDI_LB'] = 55
    # param_dict['SDI_UB'] = 85
        
    return param_dict

def make_keyfile(template, param_dict):
    filename = '../keyfiles/{}/{}.key'.format(param_dict['run_name'], param_dict['stand_id'])
    keyfile_path = os.path.abspath(filename)
    with open(keyfile_path,'w') as keyfile:
        keyfile.write(template.render(**param_dict))

    return keyfile_path

def get_keyfile_template(path_to_template):
    with open(path_to_template, 'r') as base_keyfile:
        template = Template(base_keyfile.read())
    return template


def run_fvs(params, species, stand_id):
    param_dict = make_param_dict(params, species, stand_id)
    KEYFILE_TEMPLATE = '../models/Base_Rx.key'
    template = get_keyfile_template(KEYFILE_TEMPLATE)
    keyfile = make_keyfile(template, param_dict)

    proc = subprocess.call(['/usr/local/bin/FVSnc',
                           '--keywordfile={}'.format(keyfile)],
                          stderr=subprocess.PIPE,
                          stdout=subprocess.PIPE)
    return proc

def run_score_batch(params, all_stands, species, num_cores):

    #stand_ids = np.random.choice(all_stands, sample_size)

    # args =[(params, species, run_id, id) for id in all_stands]
    # with multiprocessing.Pool(num_cores) as p:
    #     p.starmap(run_fvs, args)
    
    map_to_run = partial(run_fvs, params, species)
    with multiprocessing.Pool(num_cores) as p:
        p.map(map_to_run, all_stands)

In [131]:
stand_data = pd.read_excel('../data/groundtruth.xlsx')
all_stands = np.unique(stand_data.stand_id)

#change run name in make_param_dict
run_score_batch(super_high_mort, all_stands, 'All', 4)

In [102]:
for parent, dirnames, filenames in os.walk('../keyfiles/'):
    for fn in filenames:
        if fn.lower().endswith('.out'):
            os.remove(os.path.join(parent, fn))
        elif fn.lower().endswith('.key'):
            os.remove(os.path.join(parent, fn))

In [ ]:
# files = glob.glob('../keyfiles/*')
# for f in files:
#     os.remove(f)